지리정보 분석후 맵생성
  - 위도 경도 정보가 필요
    - GPS정보는 지오데이터 네이버지도 api에서 구할수 있음
    - 주소정보입력 - gps정보취득
    - 시각화 라이브러사용(포리움)

In [1]:
# 주소데이터

In [2]:
import pandas as pd

In [6]:
coffe_store = pd.read_csv('./CoffeeBean.csv',encoding='cp949',index_col=0)
coffe_store.head()

,store,address,phone
0,학동역 DT점,서울시 강남구 학동로 211 1층,02-3444-9973
1,수서점,서울시 강남구 광평로 280 수서동 724호,02-3412-2326
2,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615
3,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778
4,메가박스점,서울 강남구 삼성동 159 코엑스몰 지하2층,02-6002-3320


In [7]:
# https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B040A3&vw_cd=MT_ZTITLE&list_id=A_7&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=MT_ZTITLE&path=%252FstatisticsList%252FstatisticsListIndex.do

In [17]:
addrs = [addr.split() for addr in coffe_store.address]
for addr in addrs:
  if  '서울' in addr[0]:
    addr[0] = '서울특별시'
  elif '부산' in addr[0]:
    addr[0] = '부산광역시'
  elif '인천' in addr[0]:
    addr[0] = '인천광역시'
  elif '광주' in addr[0]:
    addr[0] = '광주광역시'
  elif '대전' in addr[0]:
    addr[0] = '대전광역시'
  elif '울산' in addr[0]:
    addr[0] = '울산광역시'
  elif '세종' in addr[0]:
    addr[0] = '세종특별자치시'
  elif '경기' in addr[0]:
    addr[0] = '경기도'
  elif '충북' in addr[0]:
    addr[0] = '충청북도'
  elif '충남' in addr[0]:
    addr[0] = '충청남도'
  elif '전북' in addr[0]:
    addr[0] = '전라북도'
  elif '전남' in addr[0]:
    addr[0] = '전라남도'
  elif '경북' in addr[0]:
    addr[0] = '경상북도'
  elif '경남' in addr[0]:
    addr[0] = '경상남도'
  elif '제주' in addr[0]:
    addr[0] = '제주특별자치도'
  elif '대구' in addr[0]:
    addr[0] = '대구광역시'

In [23]:
coffe_store['address2'] = [' '.join(add) for add in addrs]
coffe_store.head()

,store,address,phone,address2
0,학동역 DT점,서울시 강남구 학동로 211 1층,02-3444-9973,서울특별시 강남구 학동로 211 1층
1,수서점,서울시 강남구 광평로 280 수서동 724호,02-3412-2326,서울특별시 강남구 광평로 280 수서동 724호
2,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층
3,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층
4,메가박스점,서울 강남구 삼성동 159 코엑스몰 지하2층,02-6002-3320,서울특별시 강남구 삼성동 159 코엑스몰 지하2층


In [30]:
coffe_store.to_csv('new_address.csv',encoding='utf-8')

In [ ]:
!pip install folium

In [ ]:
import folium
map_osm = folium.Map(location=[37.512472,127.102527],zoom_start=16)
map_osm

In [27]:
map_osm.save('temp.html')

In [42]:
import requests
import numpy as np
address = '서울특별시 강남구 삼성동 159'
def find_lat_lon(address):
  try:
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        "q" : address,
        'format':"json"
    }
    response = requests.get(base_url,params=params)
    data = response.json()
    return data[0]['lat'], data[0]['lon']
  except Exception:
    return np.nan

In [ ]:
result = [find_lat_lon(address.split()[:3]) for address in coffe_store['address2'] ]
result

In [ ]:
coffe_store = pd.concat( [coffe_store,pd.DataFrame(result,columns=['lat','lon'])], axis=1 )
coffe_store.head()

In [ ]:
coffe_store[coffe_store.isnull().any(axis=1)]

In [ ]:
coffe_store = coffe_store.dropna()
coffe_store.isnull().sum()

In [67]:
# 메인지도
map_cb = folium.Map(location=[37.512472,127.102527],zoom_start=15)
# 좌표설정
for i, data in coffe_store.iterrows():
  folium.Marker(location=[data['lat'] ,data['lon'] ],
                icon=folium.Icon(color='red',icon='star'),
                popup=data['address2'],
                ).add_to(map_cb)

In [68]:
map_cb.save('coffe_store.html')

In [70]:
# 코렙에서는 안되고 로컬pc의 가상환경에서는 실제 물리적인 드라이브에 저장되므로 가능
# 코렙은 구글클라우드에서 제공하는 가상환경 이라서 안됨
import webbrowser
webbrowser.open('./coffe_store.html')

False